## TF-IDF & Word Embedding

## TF - IDF Vectorizer

In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Konfigurasi Pandas agar tampilan tabel tidak terpotong
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# --- Fungsi pembersihan teks ---
def clean_text(text):
    """
    Membersihkan teks dengan:
    - Mengubah huruf menjadi kecil
    - Menghapus tanda baca, angka, dan spasi berlebih
    """
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Membaca data dari file ---
try:
    df = pd.read_csv('berita_tempo.csv')
    print("✅ Berhasil memuat data dari 'berita_tempo.csv'")
    print(f"Jumlah baris awal: {len(df)}")
    print("Kolom yang tersedia:", df.columns.tolist())

    # --- Cek kolom wajib ---
    required_cols = ['id_berita', 'judul', 'kategori']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Kolom '{col}' tidak ditemukan dalam dataset.")

    # --- Ambil maksimal 500 data ---
    if len(df) > 500:
        df = df.head(500).copy()
        print(f"Data dibatasi ke {len(df)} baris pertama untuk efisiensi analisis.")

    # --- Bersihkan kolom judul ---
    print("\nMembersihkan teks pada kolom 'judul'...")
    df['judul_clean'] = df['judul'].fillna('').astype(str).apply(clean_text)
    print("Contoh hasil pembersihan:")
    print(df[['judul', 'judul_clean']].head())

    # --- Siapkan korpus untuk analisis ---
    corpus = df['judul_clean'].tolist()

    # --- Langkah 1: CountVectorizer ---
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

    print("\n>>> Matriks Term Frequency (CountVectorizer):")
    print(count_df.to_string())

    # --- Langkah 2: TF-IDF Transformer ---
    transformer = TfidfTransformer()
    transformer.fit(X)
    idf_values = transformer.idf_
    feature_names = vectorizer.get_feature_names_out()
    idf_df = pd.DataFrame({'word': feature_names, 'idf_value': idf_values})

    print("\n>>> Nilai IDF (Inverse Document Frequency):")
    print(idf_df.to_string())

    # --- Langkah 3: Matriks TF-IDF ---
    tfidf_matrix = transformer.transform(X)
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    print("\n>>> Matriks TF-IDF (Term Frequency - Inverse Document Frequency):")
    print(tfidf_df.to_string())

    print("\nSelesai memproses TF-IDF untuk kolom 'judul'!")
    print("-" * 80)

except FileNotFoundError:
    print("\n❌ Error: File 'berita_tempo.csv' tidak ditemukan.")
    print("Pastikan file berada di direktori yang sama dengan kode ini.")
except Exception as e:
    print(f"\n⚠️ Terjadi kesalahan: {e}")

✅ Berhasil memuat data dari 'berita_tempo.csv'
Jumlah baris awal: 149
Kolom yang tersedia: ['id_berita', 'judul', 'isi', 'kategori']

Membersihkan teks pada kolom 'judul'...
Contoh hasil pembersihan:
                                                                                        judul  \
0                               Jadwal Persib Bandung Hari Ini: Berjuang Bangkit Vs Persebaya   
1                                       Persib Bandung Pinjamkan Dimas Drajad ke Malut United   
2                         Prediksi Persib Bandung vs Persebaya pada Pekan Kelima Super League   
3                          Persija Jakarta Tunjuk Bambang Pamungkas sebagai Direktur Olahraga   
4  Hasil Super League: Persita Raih Kemenangan Perdana, Kalahkan PSM Makassar secara Dramatis   

                                                                                judul_clean  
0                              jadwal persib bandung hari ini berjuang bangkit vs persebaya  
1                            

## Word Embeding

In [8]:
import pandas as pd
import re
import numpy as np
from gensim.models import Word2Vec
import spacy
from transformers import pipeline
from tqdm import tqdm
import warnings

# Hilangkan peringatan
warnings.filterwarnings("ignore")

# Atur tampilan hasil biar full di terminal
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
np.set_printoptions(threshold=np.inf)

# === FUNGSI PEMBERSIH TEKS ===
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# === FUNGSI UTAMA ===
def process_by_category(df, text_column='judul', category_column='kategori', sample_size=500):
    print(">>> MEMPROSES BERDASARKAN KATEGORI BERITA (MAKS 500 DATA PER KATEGORI)\n")
    print(f"Kolom teks: {text_column} | Kolom kategori: {category_column}")
    print("-" * 80)

    categories = df[category_column].dropna().unique().tolist()

    # Progress bar untuk kategori
    for category in tqdm(categories, desc="Proses per Kategori", colour="green"):
        print(f"\n==============================")
        print(f"📂 KATEGORI: {category}")
        print("==============================")

        # Ambil data berdasarkan kategori
        subset = df[df[category_column] == category]
        texts = subset[text_column].dropna().astype(str).tolist()

        # Ambil maksimal 500 teks
        sampled_texts = texts[:sample_size] if len(texts) >= sample_size else texts
        print(f"✅ Jumlah data diambil: {len(sampled_texts)}")

        # Bersihkan & tokenisasi dengan progress bar
        cleaned_texts = []
        print("\nMembersihkan teks...")
        for t in tqdm(sampled_texts, desc="Cleaning teks", leave=False):
            cleaned_texts.append(clean_text(t))

        tokenized_corpus = [t.split() for t in cleaned_texts if t.strip() != '']

        # === WORD EMBEDDING ===
        print("\n>>> BAGIAN 1: WORD EMBEDDING")
        print("---")
        if tokenized_corpus:
            embedding_dim = 128
            print("Melatih model Word2Vec...")
            model = Word2Vec(
                sentences=tqdm(tokenized_corpus, desc="Training Word2Vec", leave=False),
                vector_size=embedding_dim,
                window=5,
                min_count=1,
                sg=0
            )
            model.train(tokenized_corpus, total_examples=model.corpus_count, epochs=10)

            words = list(model.wv.index_to_key)
            vectors = [model.wv[word] for word in words]
            embedding_df = pd.DataFrame(vectors, index=words)

            print(f"📊 Contoh hasil Word Embedding untuk kategori '{category}' (25 kata pertama):\n")
            print(embedding_df.head(25).to_string())
            print(f"\nTotal {len(words)} kata unik dengan {embedding_dim} dimensi.")
        else:
            print("❌ Tidak ada teks valid untuk dibuat Word2Vec.")
        print("-" * 80)

        # === ANALISIS SENTIMEN ===
        print("\n>>> BAGIAN 2: ANALISIS SENTIMEN")
        print("---")
        try:
            classifier_sentiment = pipeline(
                "sentiment-analysis",
                model="w11wo/indonesian-roberta-base-sentiment-classifier"
            )

            contoh_ulasan = [
                "Saya sangat puas dengan layanan berita ini.",
                "Berita ini terlalu negatif dan tidak objektif.",
                "Tulisan cukup informatif tapi bisa dibuat lebih netral."
            ]

            print("Menjalankan analisis sentimen pada contoh ulasan...")
            hasil = []
            for text in tqdm(contoh_ulasan, desc="Analisis Sentimen", leave=False):
                hasil.append(classifier_sentiment(text)[0])

            for i, (text, res) in enumerate(zip(contoh_ulasan, hasil), start=1):
                print(f"Ulasan {i}: '{text}'")
                print(f"  Label: {res['label']} -> Skor: {res['score']:.4f}")
        except Exception as e:
            print(f"⚠️  Gagal menjalankan analisis sentimen: {e}")
        print("-" * 80)

        # === NAMED ENTITY RECOGNITION (NER) ===
        print("\n>>> BAGIAN 3: DETEKSI ENTITAS BERNAMA (NER)")
        print("---")
        try:
            nlp = spacy.load("en_core_web_sm")
            text_sample = "Apple Inc. founded by Steve Jobs in California, is a major technology company."
            print("Menganalisis teks contoh untuk NER...")
            for _ in tqdm(range(1), desc="NER", leave=False):  # Dummy progress
                doc = nlp(text_sample)
            print(f"Teks Contoh: {text_sample}\n")
            print("Entitas yang Terdeteksi:")
            for ent in doc.ents:
                print(f"  - {ent.text} ({ent.label_})")
        except OSError:
            print("⚠️  Model spaCy belum terinstal. Jalankan:")
            print("    python -m spacy download en_core_web_sm")
        print("-" * 80)


# === BAGIAN UTAMA ===
try:
    df = pd.read_csv('berita_tempo.csv')
    print("✅ File 'berita_tempo.csv' berhasil dimuat.")
    print(f"Jumlah total data: {len(df)}")
    print(f"Kolom tersedia: {df.columns.tolist()}")
    print("-" * 80)

    # Pastikan kolom wajib ada
    required_cols = ['id_berita', 'judul', 'kategori']
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Kolom '{col}' tidak ditemukan dalam dataset.")

    process_by_category(df, text_column='judul', category_column='kategori', sample_size=500)

except FileNotFoundError:
    print("❌ File 'berita_tempo.csv' tidak ditemukan.")
    print("Pastikan file berada di folder yang sama dengan script ini.")
except Exception as e:
    print(f"⚠️ Terjadi kesalahan: {e}")

✅ File 'berita_tempo.csv' berhasil dimuat.
Jumlah total data: 149
Kolom tersedia: ['id_berita', 'judul', 'isi', 'kategori']
--------------------------------------------------------------------------------
>>> MEMPROSES BERDASARKAN KATEGORI BERITA (MAKS 500 DATA PER KATEGORI)

Kolom teks: judul | Kolom kategori: kategori
--------------------------------------------------------------------------------


Proses per Kategori:   0%|          | 0/3 [00:00<?, ?it/s]


📂 KATEGORI: bola
✅ Jumlah data diambil: 50

Membersihkan teks...



>>> BAGIAN 1: WORD EMBEDDING
---
Melatih model Word2Vec...


📊 Contoh hasil Word Embedding untuk kategori 'bola' (25 kata pertama):

                  0         1         2         3         4         5         6         7         8         9         10        11        12        13        14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        33        34        35        36        37        38        39        40        41        42        43        44        45        46        47        48        49        50        51        52        53        54        55        56        57        58        59        60        61        62        63        64        65        66        67        68        69        70        71        72        73        74        75        76        77        78        79        80        81        82        83        84        85        86        87        88        89        90        

...

Device set to use cpu


Menjalankan analisis sentimen pada contoh ulasan...


Ulasan 1: 'Saya sangat puas dengan layanan berita ini.'
  Label: positive -> Skor: 0.9973
Ulasan 2: 'Berita ini terlalu negatif dan tidak objektif.'
  Label: negative -> Skor: 0.9993
Ulasan 3: 'Tulisan cukup informatif tapi bisa dibuat lebih netral.'
  Label: negative -> Skor: 0.7540
--------------------------------------------------------------------------------

>>> BAGIAN 3: DETEKSI ENTITAS BERNAMA (NER)
---
Menganalisis teks contoh untuk NER...


Proses per Kategori: 100%|██████████| 3/3 [03:08<00:00, 62.97s/it]

Teks Contoh: Apple Inc. founded by Steve Jobs in California, is a major technology company.

Entitas yang Terdeteksi:
  - Apple Inc. (ORG)
  - Steve Jobs (PERSON)
  - California (GPE)
--------------------------------------------------------------------------------


## Klasifikasi

In [9]:
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import sys
import warnings

# Hilangkan warning TensorFlow dan Pandas
warnings.filterwarnings("ignore")

# Konfigurasi Pandas dan TensorFlow untuk hasil yang konsisten
np.random.seed(100)
tf.random.set_seed(100)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 150)
pd.set_option('display.max_colwidth', None)

# --- FUNGSI PEMBERSIH TEKS ---
def clean_text(text):
    """Membersihkan teks dari tanda baca, angka, dan ubah ke huruf kecil."""
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


# --- MEMUAT DATA CSV ---
file_path = 'berita_tempo.csv'

try:
    df = pd.read_csv(file_path)
    print(f"✅ Berhasil memuat data dari: {file_path}")

    # Ambil hanya kolom yang dibutuhkan
    TEXT_COLUMN = 'judul'
    LABEL_COLUMN = 'kategori'

    # Validasi kolom
    for col in [TEXT_COLUMN, LABEL_COLUMN]:
        if col not in df.columns:
            print(f"❌ Error: Kolom '{col}' tidak ditemukan.")
            print("Kolom yang tersedia:", df.columns.tolist())
            sys.exit()

    # Ambil maksimal 500 data untuk efisiensi
    df = df.head(500)
    print(f"📊 Data dipotong menjadi {len(df)} baris pertama untuk analisis.")

    # Bersihkan data kosong
    df = df.dropna(subset=[TEXT_COLUMN, LABEL_COLUMN])
    df[LABEL_COLUMN] = df[LABEL_COLUMN].astype(str).str.strip()
    df = df[df[LABEL_COLUMN] != '']

    if df.empty:
        print("❌ Tidak ada data yang valid setelah pembersihan.")
        sys.exit()

    corpus = df[TEXT_COLUMN].apply(clean_text).tolist()

    # Ubah label kategori menjadi angka
    label_to_id = {label: i for i, label in enumerate(df[LABEL_COLUMN].unique())}
    id_to_label = {i: label for label, i in label_to_id.items()}
    labels = [label_to_id[label] for label in df[LABEL_COLUMN]]

    print(f"\n📁 Jumlah kategori unik: {len(label_to_id)}")
    print(f"Kategori berita: {list(label_to_id.keys())}")

    if len(label_to_id) < 2:
        print("❌ Error: Hanya ada satu kategori, klasifikasi tidak bisa dilakukan.")
        sys.exit()

except FileNotFoundError:
    print(f"❌ File '{file_path}' tidak ditemukan.")
    print("Pastikan file berada di folder yang sama dengan script ini.")
    sys.exit()
except Exception as e:
    print(f"⚠️ Terjadi kesalahan saat membaca data: {e}")
    sys.exit()


# --- Distribusi kategori ---
print("\n--- Distribusi Kategori ---")
print(df[LABEL_COLUMN].value_counts().to_string())

# Cek kategori dengan hanya 1 data
rare_cats = df[LABEL_COLUMN].value_counts()[df[LABEL_COLUMN].value_counts() == 1].index.tolist()
if rare_cats:
    print(f"\n⚠️ Peringatan: Kategori dengan hanya 1 sampel: {rare_cats}")
    print("Kategori ini dapat menyebabkan error saat pembagian data stratified.")

# --- Split data (train/test) ---
try:
    X_train, X_test, y_train, y_test = train_test_split(
        corpus, labels, test_size=0.2, random_state=100, stratify=labels
    )
    print("\n✅ Data berhasil dibagi secara stratified.")
except ValueError:
    print("\n⚠️ Stratified split gagal (karena kategori dengan hanya 1 data). Menggunakan split normal.")
    X_train, X_test, y_train, y_test = train_test_split(
        corpus, labels, test_size=0.2, random_state=100
    )

print(f"Jumlah data latih: {len(X_train)} | Jumlah data uji: {len(X_test)}")

results = {}  # Untuk menyimpan hasil akurasi


# ======================================================================
# 1️⃣ TF-IDF + LOGISTIC REGRESSION
# ======================================================================
print("\n" + "="*90)
print(">>> 1️⃣ KLASIFIKASI MENGGUNAKAN TF-IDF (LOGISTIC REGRESSION)")
print("="*90)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Jumlah fitur unik: {X_train_tfidf.shape[1]}")

model_tfidf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=100)
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
acc_tfidf = accuracy_score(y_test, y_pred_tfidf)
results["TF-IDF"] = acc_tfidf

print(f"\n🎯 Akurasi TF-IDF: {acc_tfidf:.4f}")
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_tfidf, target_names=[id_to_label[i] for i in sorted(id_to_label.keys())], zero_division=0))


# ======================================================================
# 2️⃣ WORD EMBEDDING + NEURAL NETWORK
# ======================================================================
print("\n" + "="*90)
print(">>> 2️⃣ KLASIFIKASI MENGGUNAKAN WORD EMBEDDING (NEURAL NETWORK)")
print("="*90)

tokenizer = Tokenizer(num_words=5000, oov_token="<unk>")
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(t.split()) for t in corpus)
num_classes = len(set(labels))

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

embedding_dim = 100

model_emb = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model_emb.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("\n🔁 Melatih model Word Embedding (10 epochs)...")

history = model_emb.fit(
    X_train_pad, np.array(y_train),
    validation_split=0.1,
    epochs=10,
    verbose=1
)

loss, acc_emb = model_emb.evaluate(X_test_pad, np.array(y_test), verbose=0)
results["Word Embedding"] = acc_emb

print(f"\n🎯 Akurasi Word Embedding: {acc_emb:.4f}")
y_pred_probs = model_emb.predict(X_test_pad, verbose=0)
y_pred_emb = np.argmax(y_pred_probs, axis=1)

print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred_emb, target_names=[id_to_label[i] for i in sorted(id_to_label.keys())], zero_division=0))


# ======================================================================
# 🔍 PERBANDINGAN HASIL
# ======================================================================
print("\n" + "#"*90)
print(">>> 🔍 PERBANDINGAN HASIL KEDUA METODE <<<")
print("#"*90)

print(f"Akurasi TF-IDF (Logistic Regression): {results['TF-IDF']:.4f}")
print(f"Akurasi Word Embedding (Neural Network): {results['Word Embedding']:.4f}")

print("\n📚 PERBANDINGAN:")
print("• TF-IDF merepresentasikan kata berdasarkan frekuensi dan kelangkaan (tanpa konteks).")
print("• Word Embedding menangkap makna semantik antar kata dan konteks kalimat.")
print("• TF-IDF lebih cepat, Word Embedding lebih kuat untuk data besar dan konteks kompleks.")

if results['Word Embedding'] > results['TF-IDF']:
    print("\n✅ Kesimpulan: Word Embedding unggul — model memahami konteks antar kata lebih baik.")
elif results['TF-IDF'] > results['Word Embedding']:
    print("\n✅ Kesimpulan: TF-IDF unggul — kategori berita dibedakan kuat oleh kata kunci spesifik.")
else:
    print("\n✅ Kesimpulan: Keduanya seimbang — TF-IDF cukup efektif untuk teks pendek seperti judul.")
print("#"*90)


✅ Berhasil memuat data dari: berita_tempo.csv
📊 Data dipotong menjadi 149 baris pertama untuk analisis.

📁 Jumlah kategori unik: 3
Kategori berita: ['bola', 'metro', 'dunia']

--- Distribusi Kategori ---
kategori
bola     50
dunia    50
metro    49

✅ Data berhasil dibagi secara stratified.
Jumlah data latih: 119 | Jumlah data uji: 30

>>> 1️⃣ KLASIFIKASI MENGGUNAKAN TF-IDF (LOGISTIC REGRESSION)
Jumlah fitur unik: 591

🎯 Akurasi TF-IDF: 0.8667

Laporan Klasifikasi:
              precision    recall  f1-score   support

        bola       0.82      0.90      0.86        10
       metro       1.00      0.90      0.95        10
       dunia       0.80      0.80      0.80        10

    accuracy                           0.87        30
   macro avg       0.87      0.87      0.87        30
weighted avg       0.87      0.87      0.87        30


>>> 2️⃣ KLASIFIKASI MENGGUNAKAN WORD EMBEDDING (NEURAL NETWORK)

🔁 Melatih model Word Embedding (10 epochs)...
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2